# TD 6: Les élections

Dans cette activité on va utiliser des listes, tuples et dictionnaires pour compter des votes et déterminer le vainqueur d'une élection.

## Tutoriel: le vote à majorité simple

On veut d'abord implémenter un des algorithmes de vote les plus simples, la règle dite de "pluralité", c'est-à-dire à majorité simple -- le système utilisé au Canada (et dans plusieurs autres pays) pour élire les députés.

Les candidats à l'élection sont V. Poutchine, J. Troudeau, et E. Maqueron.

Dans le contexte du vote à majorité simple, un bulletin de vote pourrait être représenté par un seul nom. Cependant, en anticipant les autres mécanismes de vote qu'on va explorer par la suite, on va représenter les préférences complètes de chaque électeur par la liste des candidats dans l'ordre des préférences de l'électeur: d'abord son premier choix, ensuite son deuxième choix, etc.

Par exemple, pour les préférences d'un électeur qui préfère M. Troudeau aux autres, et préfère M. Maqueron à M. Poutchine, on aurait la liste ```["Troudeau", "Maqueron", "Poutchine"]```. Dans la suite on appellera une telle liste un _profil de préférences_.

Les préférences de la population peuvent se représenter par un grand nombre de listes de cette forme, une par électeur. Pour simplifier la représentation, on va représenter cet ensemble de préférences par un ensemble de tuples, qui donne pour chaque profil de préférences le nombre d'électeurs auquel correspond ce profil de préférences:

In [1]:
preferences = [(["Troudeau", "Maqueron", "Poutchine"], 43), (["Poutchine", "Troudeau", "Maqueron"], 61), (["Maqueron", "Troudeau", "Poutchine"], 28), (["Maqueron", "Poutchine", "Troudeau" ], 9), (["Troudeau", "Poutchine", "Maqueron"], 7)]

Cette liste nous dit que 43 électeurs préfèrent M. Troudeau, devant M. Maqueron en deuxième place, et M. Poutchine en troisième, que 61 électeurs ont le profil avec M. Poutchine en premier, suivi de M. Troudeau puis M. Maqueron, etc.

Pour comptabiliser les votes, on va créer un dicitonnaire avec comme clés les noms des candidats, et comme valeurs le nombre de votes qu'ils reçoivent. On commence par créer ce dictionnaire vide.

In [2]:
votes = {}

On va ensuite devoir parcourir la liste de profils de préférences, et en supposant que les électeurs vont voter pour leur candidat favori, comptabiliser les votes que recevront les différents candidats.

Chaque élément de la liste est une paire _(profil de préférences, nombre d'électeurs)_. Prenons un exemple et séparons les deux éléments:

In [3]:
paire = (["Troudeau", "Maqueron", "Poutchine"], 43)
prefs = paire[0]
votants = paire[1]

In [4]:
prefs

['Troudeau', 'Maqueron', 'Poutchine']

In [5]:
votants

43

On peut maintenant extraire le candidat favori, qui est le premier élément de ```prefs```:

In [6]:
prefs[0]

'Troudeau'

Il faudra donc ajouter 43 voix pour M. Troudeau.

Voyons si ce candidat a déjà des votes:

In [7]:
"Troudeau" in votes

False

Il n'en a pas, on peut donc simplement lui attribuer les 43 votes comme ceci:

In [8]:
votes["Troudeau"] = 43

Autrement dit en utilisant les variables appropriées:

In [9]:
votes[prefs[0]] = votants

Si un candidat avait déjà des voix (comme M. Troudeau après cette dernière opération), il faudrait faire la somme des voix déjà comptabilisées avec celles données par le tuples actuel:

In [10]:
votes[prefs[0]]

43

Supposons qu'on comptabilise un nouveau profil de préférences avec d'autres voix allant à M. Troudeau:

In [11]:
paire = (["Troudeau", "Poutchine", "Maqueron"], 7)
prefs, votants = paire # on peut aussi affecter directement les deux variables

On calcule le nouveau total de votes, et on met à jour le dictionnaire:

In [12]:
nouveau_total = votes[prefs[0]] + votants
votes[prefs[0]] = nouveau_total

In [13]:
votes

{'Troudeau': 50}

Les 7 nouveaux votes on bien été comptabilisés.
À noter qu'on aurait pu écrire simplement ```votes[prefs[0]] = votes[prefs[0]] + votants```, sans utiliser la variable ```nouveau_total```.

Reprenons donc notre dictionnaire de votes et répétons cete opération pour tous les profils de préférences listés:

In [14]:
votes = {}
for paire in preferences:
    prefs, votants = paire[0], paire[1]
    favori = prefs[0]
    if (favori in votes):
        votes[favori] = votes[favori]+ votants # on met à jour s'il y avait déjà des voix
    else:
        votes[favori] = votants 

Et voilà, nous avons comptabilisé les votes:

In [15]:
votes

{'Troudeau': 50, 'Poutchine': 61, 'Maqueron': 37}

M. Poutchine a donc remporté cette élection. Afin de trouver ça de manière automatisée, il suffit de parcourir les éléments du dictionnaire et de trouver la clé dont la valeur associée est la plus élevée.

Pour parcourir la liste de candidats, on énumère simplement les clés du dictionnaire: ceci nous donne les candidats ayant reçu au moins un vote.

In [16]:
for candidat in votes:
    print(candidat,":", votes[candidat], "voix")

Troudeau : 50 voix
Poutchine : 61 voix
Maqueron : 37 voix


Afin de trouver le gagnant de l'élection (la clé avec la valeur la plus élevée), on va adapter une "recette" vue dans le cours sur les boucles, pour trouver l'élément "le plus..." dans un ensemble. La recette est comme suit: on commence par choisir comme "candidat" initial la première valeur de l'ensemble, et ensuite énumère les autres valeurs à la recherche d'une "meilleure" valeur selon notre critère, c'est à dire plus haute, plus basse, etc. 

Dans le cas présent, il est un peu compliqué d'accéder au "premier" élément du dictionnaire, alors pour suivre précisément la recette il faudrait en choisir un au hasard, ou imposer un ordre... plus simplement, ici on cherche le candidat avec le maximum de votes, et comme on sait que le gagnant de l'élection aura au moins un vote, on va initialiser notre maximum temporaire à zéro, plutôt que de prendre le nombre de votes du "premier" candidat.

On a donc l'algorithme suivant:

In [17]:
nbvotes_max = 0 #valeur candidate initiale
for candidat in votes: #ici candidat se réfère à l';election, pas à la "valeur candidate" de la recette
    if(votes[candidat]>nbvotes_max):
        nbvotes_max = votes[candidat]
        meilleur = candidat     # il faut retenir qui a reçu ce nombre de votes

In [18]:
meilleur, nbvotes_max

('Poutchine', 61)

M. Poutchine a bien remporté l'élection.

Remarque: on suppose ici qu'il n'y a pas eu d'égalité entre deux candidats.

## Exercice guidé: le deuxième tour

Dans cet exercice on va implémenter la règle vote de la "pluralité à deux tours", c'est à dire qu'à l'issue d'un premier scrutin, si un candidat a la majorité absolue des voix (au moins 50% + 1 voix), il est élu, et sinon on garde les deux candidats ayant reçu le plus de voix et on organise un deuxième tour, où les électeurs départagent seulement ces deux candidats.

__2.1__ Reprendre le code qui trouve le vainqueur de l'élection, et ajouter un comptage du total des voix, afin de pouvoir vérifier si le gagnant a la majorité absolue.

__2.2__ Ajouter du code pour déclarer le candidat élu dans le cas où il obtient la majorité absolue.

__2.3__ Dans le cas où aucun candidat n'a la majorité absolue, il va falloir un deuxième tour, et pour cela, savoir quels sont les deux candidats ayant reçu le plus de voix. Dans notre exemple ce sont MM. Poutchine et Troudeau, mais on veut trouver cela automatiquement.

Modifier le code qui calcule le résultat du premier tour, pour trouver _les deux_ candidats ayant le plus de votes, plutôt que le _seul_ gagnant. 

Créer une paire (un tuple à deux éléments) avec les deux candidats du deuxième tour. 

Pour calculer le résultat du deuxième tour, on va supposer que les électeurs votent pour le candidat qu'ils préfèrent entre les deux. 

__2.4__ En reprenant le code qui comptabilise les votes, trouver pour chaque profil de préférences le candidat qui recevra le vote. Attention: le code doit être correct quel que soit le nombre de candidats dans le profil de préférences.

__2.5__ Reprendre l'ensemble du code pour permettre l'égalité de voix entre deux candidats: il faut principalement modifier le décompte des voix au premier tour, qui doit permettre de trouver des _listes_ de candidats avec le même nombre de voix. Si on a une situation où il n'y a pas exactement deux candidats en tête, on départage les candidats à égalité en choisissant au hasard.

## Exercice non guidé: méthodes de Borda et méthode de Copeland

Le but de l'exercice est d'implémenter deux autres règles de vote: la technique de Borda et la technique de Copeland.

La méthode de Borda est une méthode à base de points: en supposant qu'il y a _n_ candidats, pour chaque électeur on regarde ses préférences, et on donne _n_ points pour son candidat favori, _n-1_ points pour le deuxième, n-2 pour le troisième, et ainsi de suite jusqu à 1 point pour le dernier. Les candidats sont ensuites classés selon leur score: le candidat avec le plus de points est élu.

La méthode de Copeland consiste à comparer les candidats deux-à-deux, comme s'ils avaient atteint ensemble le second tour d'une élection à deux tours. Pour chaque "duel" on donne +1 au vainqueur, 0 au vaincu. S'il y a égalité, un demi-point chacun. Après cela on compte les points de chaque candidat, et on choisit le candidat avec le plus de points. S'il y a égalité entre deux candidats, il y a plusieurs manières de départager les candidats ex-aequo: en général on se tourne vers une autre règle de vote, comme par exemple la méthode de Borda.
 